# Norwegian Salmon Health Crisis: 20 Years of Disease vs Production Expansion (2005-2024)

##  Executive Summary
This analysis examines two decades of disease surveillance in Norwegian Atlantic salmon aquaculture, revealing critical insights about the relationship between industrial expansion and fish health. Despite a 42.5% increase in production licenses, disease patterns show complex dynamics; some infections declined dramatically due to vaccines while others increased, highlighting both successes and ongoing challenges in sustainable aquaculture.

##  Project Objectives
1. **Quantify** the relationship between production expansion and disease incidence
2. **Evaluate** the impact of vaccine introductions on target diseases  
3. **Identify** emerging disease threats in modern salmon farming
4. **Provide** data-driven recommendations for sustainable aquaculture

##  Dataset Overview
**Source:** Norwegian Directorate of Fisheries and Veterinary Institute  
**Period:** 2005-2024 (20 continuous years)  
**Records:** Annual disease case counts + production licenses  
**Diseases Tracked:** 8 major salmon pathogens  
**File:** `salmon_diseases_2005_2024.csv`

### Key Variables:
- **Production Indicator:** Annual operational licenses
- **Diseases Monitored:**
  - ISA (Infectious Salmon Anaemia)
  - IPN (Infectious Pancreatic Necrosis)
  - PD (Pancreas Disease)
  - HSMI (Heart and Skeletal Muscle Inflammation)
  - Furunculosis
  - BKD (Bacterial Kidney Disease)
  - CMS (Cardiomyopathic Syndrome)
  - Cold-water vibriosis

##  Methodology

### Data Processing Pipeline
1. **Data Cleaning:** Handle missing values, convert data types
2. **Feature Engineering:** Calculate derived metrics
3. **Exploratory Analysis:** Visual trends and correlations
4. **Statistical Modeling:** Regression and hypothesis testing
5. **Interpretation:** Translate findings into insights

### Analytical Techniques
- **Descriptive Statistics:** Temporal trends and distributions
- **Correlation Analysis:** Disease-disease and disease-license relationships
- **Regression Models:** Linear, polynomial, and regularized regression
- **Comparative Analysis:** Pre/post vaccine periods
- **Time Series Analysis:** Lag effects and temporal patterns

##  Key Analyses Performed

### 1. Disease Trend Visualization
- Individual disease trajectories over 20 years
- Peak incidence identification for each pathogen
- Comparative burden assessment

### 2. Production-Disease Relationship
- License growth vs. disease incidence correlation
- Cases per license density metric
- Time-lagged effects analysis

### 3. Vaccine Impact Assessment
- IPN vaccine (introduced ~2012) effectiveness
- PD vaccine (introduced ~2017) outcomes
- Statistical significance testing of reductions

### 4. Multivariate Analysis
- Multiple regression: Total cases = f(Licenses, Year, Licenses²)
- Regularized regression for feature importance
- Residual diagnostics and model validation

##  Key Findings

### Success Stories:
- **IPN Dramatic Reduction:** Cases dropped from 200+ to <30 annually post-2012 vaccination
- **Effective Disease Management:** Overall cases per license trend shows improvement

### Emerging Concerns:
- **CMS Increase:** Steady rise from ~70 to 150+ cases despite controls
- **HSMI Variability:** Fluctuations suggest density-dependent challenges
- **Recent BKD Resurgence:** Increase in 2023-2024 warrants attention

### Production-Disease Insights:
- **Positive Correlation:** CMS and HSMI scale with production
- **Decoupled Success:** IPN reduced despite production growth (vaccine effect)
- **Mixed Patterns:** Other diseases show complex, non-linear relationships

##  Practical Implications

### For Salmon Farmers:
1. **Vaccine Priority:** Maintain IPN vaccination, consider PD vaccine adoption
2. **Density Management:** Monitor CMS and HSMI in high-density operations
3. **Surveillance Focus:** Enhanced monitoring for BKD resurgence

### For Policymakers:
1. **Vaccine Support:** Continue funding for effective immunization programs
2. **Sustainable Growth:** Balance production expansion with health safeguards
3. **Research Investment:** Target diseases without effective vaccines (CMS, BKD)

### For Industry Stakeholders:
1. **Risk Assessment:** Consider disease burden in expansion planning
2. **Technology Adoption:** Invest in advanced health monitoring systems
3. **Collaborative Solutions:** Industry-wide disease management strategies

##  Results 
The analysis generates multiple outputs:

### Visualizations:
1. `disease_trends.png` - Individual disease trajectories
2. `licenses_vs_cases.png` - Production-disease relationship
3. `correlation_heatmap.png` - Disease interrelationships
3. `disease_contribution.png` - Relative burden of each disease
4. Regression diagnostics and model visualizations

### Data Outputs:
1. `processed_salmon_diseases_data.csv` - Cleaned dataset with calculated metrics
2. `disease_summary_statistics.csv` - Aggregated statistics for each disease

##  Limitations & Considerations

### Data Constraints:
- Licenses as production proxy (not actual biomass)
- Diagnostic improvements over time may affect case detection
- Environmental factors not included in analysis
- Possible underreporting in early years

### Analytical Limitations:
- Small temporal sample (n=20 years)
- Correlation does not imply causation
- Multiple confounding factors in real world setting
- No control group for natural experiment analysis



### Prerequisites:
Python 3.8+
Required libraries: pandas, numpy, matplotlib, seaborn, scikit-learn, statsmodels

In [8]:
!pip install pandas numpy matplotlib seaborn scipy scikit-learn statsmodels ipykernel ipywidgets

  Using cached joblib-1.5.3-py3-none-any.whl.metadata (5.5 kB)
  Using cached threadpoolctl-3.6.0-py3-none-any.whl.metadata (13 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.1/8.1 MB 11.3 MB/s  0:00:00 eta 0:00:01
Using cached joblib-1.5.3-py3-none-any.whl (309 kB)
Using cached threadpoolctl-3.6.0-py3-none-any.whl (18 kB)
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3/3 [scikit-learn] [scikit-learn]


In [9]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from sklearn.linear_model import LinearRegression, Ridge, Lasso
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import r2_score, mean_squared_error, mean_absolute_error
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

plt.style.use('seaborn-v0_8-darkgrid')
sns.set_palette("husl")
%matplotlib inline
import os


In [10]:
df = pd.read_csv('data/salmon_diseases_2005_2024.csv')

print("=" * 60)
print("DATA OVERVIEW")
print("=" * 60)
print(f"Dataset shape: {df.shape}")
print(f"\nColumns: {list(df.columns)}")

print(f"\nYear column type: {type(df['Year'].iloc[0])}")
print(f"Sample Year values: {df['Year'].head().tolist()}")

try:
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df['Year'] = df['Year'].astype(int) 
    min_year = int(df['Year'].min())
    max_year = int(df['Year'].max())
    print(f"\n✅ Time period: {min_year} - {max_year}")
    print(f"✅ Number of years: {df['Year'].nunique()}")
    
except Exception as e:
    print(f"\n⚠️ Warning converting Year: {e}")
    df['Year'] = df['Year'].astype(str).str.extract('(\d+)')[0]
    df['Year'] = pd.to_numeric(df['Year'], errors='coerce')
    df['Year'] = df['Year'].astype(int) 
    
    min_year = int(df['Year'].min())
    max_year = int(df['Year'].max())
    print(f"\n✅ Time period (fallback): {min_year} - {max_year}")
    print(f"✅ Number of years: {df['Year'].nunique()}")

# Verify
print(f"\n✅ Final Year values (first 10): {df['Year'].head(10).tolist()}")
print(f"✅ Year data type: {df['Year'].dtype}")

print("\nFirst 5 rows:")
print(df.head())

print("\nMissing values:")
print(df.isnull().sum())

print("\nDescriptive statistics:")
for col in df.columns:
    if col != 'Year':  
        df[col] = pd.to_numeric(df[col], errors='coerce')

print(df.describe())

FileNotFoundError: [Errno 2] No such file or directory: 'data/salmon_diseases_2005_2024.csv'

In [ ]:
df['Year'] = df['Year'].astype(int) 

disease_cols = [col for col in df.columns if col not in ['Year', 'Licenses']]
df['Total_Cases'] = df[disease_cols].sum(axis=1)
df['Cases_per_License'] = df['Total_Cases'] / df['Licenses']
df['Production_Index'] = df['Licenses'] / df['Licenses'].iloc[0] * 100

plt.figure(figsize=(16, 12))
major_diseases = ['ISA', 'IPN', 'PD', 'HSMI', 'CMS']
colors = plt.cm.Set2(np.linspace(0, 1, len(major_diseases)))

for i, disease in enumerate(major_diseases):
    plt.subplot(3, 2, i+1)
    plt.plot(df['Year'], df[disease], marker='o', linewidth=2, color=colors[i])
    plt.xticks(df['Year'][::2])  
    plt.title(f'{disease} Cases Over Time', fontsize=14, fontweight='bold')
    plt.xlabel('Year')
    plt.ylabel('Number of Cases')
    plt.grid(True, alpha=0.3)
    max_year = df.loc[df[disease].idxmax(), 'Year']
    plt.axvline(x=max_year, color=colors[i], linestyle='--', alpha=0.5)
    plt.text(max_year, df[disease].max(), f' Peak: {max_year}', 
             verticalalignment='bottom', fontsize=9)

plt.tight_layout()
plt.savefig('disease_trends.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df['Year'] = df['Year'].astype(int)
fig, ax1 = plt.subplots(figsize=(14, 7))

ax1.plot(df['Year'], df['Licenses'], 'b-', linewidth=3, marker='s', 
         markersize=8, label='Licenses')
ax1.set_xticks(df['Year'][::2])

ax1.set_xlabel('Year', fontsize=12)
ax1.set_ylabel('Number of Licenses', color='b', fontsize=12)
ax1.tick_params(axis='y', labelcolor='b')
ax1.set_title('Production Licenses vs. Total Disease Cases (2005-2024)', 
              fontsize=16, fontweight='bold')
ax2 = ax1.twinx()
ax2.plot(df['Year'], df['Total_Cases'], 'r-', linewidth=3, marker='o', 
         markersize=8, label='Total Cases')
ax2.set_ylabel('Total Disease Cases', color='r', fontsize=12)
ax2.tick_params(axis='y', labelcolor='r')

correlation = df['Licenses'].corr(df['Total_Cases'])
ax1.text(0.02, 0.98, f'Correlation: {correlation:.3f}', 
         transform=ax1.transAxes, fontsize=12,
         verticalalignment='top',
         bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))

fig.tight_layout()
plt.savefig('licenses_vs_cases.png', dpi=300, bbox_inches='tight')
plt.show()


In [ ]:
df['Year'] = df['Year'].astype(int)
disease_cols = [col for col in df.columns if col not in ['Year', 'Licenses']]
df['Total_Cases'] = df[disease_cols].sum(axis=1)
df['Cases_per_License'] = df['Total_Cases'] / df['Licenses']

plt.figure(figsize=(14, 7))

plt.plot(df['Year'], df['Cases_per_License'], marker='o', linewidth=3, 
         color='green', markersize=8)
plt.fill_between(df['Year'], df['Cases_per_License'], alpha=0.3, color='green')

plt.xticks(df['Year'][::2])  

plt.title('Disease Cases per License (2005-2024)', fontsize=16, fontweight='bold')
plt.xlabel('Year', fontsize=12)
plt.ylabel('Cases per License', fontsize=12)
plt.grid(True, alpha=0.3)

try:
    z = np.polyfit(df['Year'], df['Cases_per_License'].fillna(0), 1)
    p = np.poly1d(z)
    plt.plot(df['Year'], p(df['Year']), "r--", linewidth=2, 
             label=f'Trend: y={z[0]:.4f}x + {z[1]:.2f}')
    plt.legend(fontsize=11)
    
    trend_direction = "increasing" if z[0] > 0 else "decreasing"
    trend_text = f"Trend: {trend_direction} ({abs(z[0]):.4f} per year)"
    
    plt.text(0.02, 0.95, trend_text, transform=plt.gca().transAxes,
             fontsize=12, verticalalignment='top',
             bbox=dict(boxstyle='round', facecolor='wheat', alpha=0.5))
    
except Exception as e:
    print(f"Trend line skipped: {e}")

for i, year in enumerate(df['Year']):
    if i % 3 == 0: 
        plt.annotate(str(year), 
                    (df['Year'].iloc[i], df['Cases_per_License'].iloc[i]),
                    xytext=(5, 5), textcoords='offset points',
                    fontsize=9, alpha=0.7)

plt.tight_layout()
plt.savefig('cases_per_license.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
vaccine_periods = {
    'IPN_vaccine': 2012,  
    'PD_vaccine': 2017,  
}

print("=" * 60)
print("VACCINE IMPACT ANALYSIS")
print("=" * 60)

for vaccine, year in vaccine_periods.items():
    disease = vaccine.split('_')[0]  
    
    if disease in df.columns:
        before = df[df['Year'] < year][disease].mean()
        after = df[df['Year'] >= year][disease].mean()
        change = ((after - before) / before) * 100
        
        print(f"\n{disease} (Vaccine ~{year}):")
        print(f"  Before ({df['Year'].min()}-{year-1}): {before:.1f} avg cases")
        print(f"  After ({year}-{df['Year'].max()}): {after:.1f} avg cases")
        print(f"  Change: {change:.1f}%")
        
        # Statistical test
        t_stat, p_value = stats.ttest_ind(
            df[df['Year'] < year][disease], 
            df[df['Year'] >= year][disease]
        )
        print(f"  t-test p-value: {p_value:.4f}")
        print(f"  Significant (p<0.05): {'Yes' if p_value < 0.05 else 'No'}")


In [ ]:
corr_matrix = df[disease_cols + ['Licenses']].corr()

plt.figure(figsize=(12, 10))
mask = np.triu(np.ones_like(corr_matrix, dtype=bool))
sns.heatmap(corr_matrix, mask=mask, annot=True, cmap='coolwarm', 
            center=0, fmt='.2f', square=True, linewidths=1)

plt.title('Correlation Matrix: Diseases and Licenses', fontsize=16, fontweight='bold')
plt.tight_layout()
plt.savefig('correlation_heatmap.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
total_cases_all_years = df[disease_cols].sum().sum()
disease_contribution = df[disease_cols].sum() / total_cases_all_years * 100

plt.figure(figsize=(12, 8))
disease_contribution.sort_values(ascending=False).plot(kind='bar', color='steelblue')
plt.title('Percentage Contribution of Each Disease (2005-2024)', 
          fontsize=16, fontweight='bold')
plt.xlabel('Disease')
plt.ylabel('Percentage of Total Cases (%)')
plt.xticks(rotation=45)
plt.grid(axis='y', alpha=0.3)

for i, v in enumerate(disease_contribution.sort_values(ascending=False)):
    plt.text(i, v + 0.5, f'{v:.1f}%', ha='center', fontweight='bold')

plt.tight_layout()
plt.savefig('disease_contribution.png', dpi=300, bbox_inches='tight')
plt.show()

In [ ]:
df_periods = pd.DataFrame({
    'Period': ['Early (2005-2010)', 'Middle (2011-2017)', 'Recent (2018-2024)'],
    'Years': ['2005-2010', '2011-2017', '2018-2024'],
    'Start_Year': [2005, 2011, 2018],
    'End_Year': [2010, 2017, 2024]
})

period_data = []

for _, period in df_periods.iterrows():
    mask = (df['Year'] >= period['Start_Year']) & (df['Year'] <= period['End_Year'])
    period_df_subset = df.loc[mask, disease_cols + ['Licenses', 'Total_Cases', 'Cases_per_License']]
    period_avg = period_df_subset.mean().to_dict()  
    period_avg['Period'] = period['Period']
    period_avg['Year_Range'] = period['Years']
    
    period_data.append(period_avg)

period_df = pd.DataFrame(period_data)

print("\n" + "=" * 60)
print("PERIOD COMPARISON")
print("=" * 60)
display_cols = ['Period', 'Year_Range', 'Licenses', 'Total_Cases', 'Cases_per_License']
display_cols = [col for col in display_cols if col in period_df.columns]

for disease in major_diseases:
    if disease in period_df.columns:
        display_cols.append(disease)

print(period_df[display_cols].round(2))

In [ ]:
print("\n SIMPLE LINEAR REGRESSION (Each Disease vs Licenses)")
print("-" * 50)

df_clean = df.copy()
df_clean = df_clean.fillna(0)  # Replace NaN with 0 for disease counts

regression_results = []
X = df_clean['Licenses'].values.reshape(-1, 1)
scaler_X = StandardScaler()
X_scaled = scaler_X.fit_transform(X)

for disease in disease_cols:
    if disease not in df_clean.columns:
        print(f"Skipping {disease} - column not found")
        continue
    
    y = df_clean[disease].values
    
    # Skip if all values are NaN or zero
    if np.all(np.isnan(y)) or np.all(y == 0):
        print(f"Skipping {disease} - all values are zero or NaN")
        continue
    y = np.nan_to_num(y, nan=0.0)
    scaler_y = StandardScaler()
    y_scaled = scaler_y.fit_transform(y.reshape(-1, 1)).flatten()
    
    model = LinearRegression()
    model.fit(X_scaled, y_scaled)
    y_pred = model.predict(X_scaled)
    r2 = r2_score(y_scaled, y_pred)
    slope = model.coef_[0]
    X_sm = sm.add_constant(X_scaled)
    model_sm = sm.OLS(y_scaled, X_sm).fit()
    p_value = model_sm.pvalues[1]
    direction = "increases" if slope > 0 else "decreases"
    
    regression_results.append({
        'Disease': disease,
        'Slope': slope,
        'R2': r2,
        'P_Value': p_value,
        'Significant': p_value < 0.05,
        'Interpretation': f"For each SD in licenses, {disease} {direction} by {abs(slope):.3f} SD"
    })

    significance = "✓" if p_value < 0.05 else "✗"
    print(f"\n{disease}:")
    print(f"  Slope (β): {slope:.4f}")
    print(f"  R²: {r2:.4f}")
    print(f"  p-value: {p_value:.4f} {significance}")
if regression_results:
    results_df = pd.DataFrame(regression_results)
    results_df = results_df.sort_values('R2', ascending=False)
    
    print("\n" + "=" * 60)
    print("📊 REGRESSION RESULTS SUMMARY (Sorted by R²)")
    print("=" * 60)
    print(results_df[['Disease', 'Slope', 'R2', 'P_Value', 'Significant']].round(4).to_string(index=False))
else:
    print("\nNo valid regression results to display.")

In [ ]:
df_clean = df.fillna(0)

if 'results_df' in locals() and len(results_df) > 0:
    top_diseases = results_df.nlargest(4, 'R2')['Disease'].tolist()
    
    fig, axes = plt.subplots(2, 2, figsize=(14, 10))
    axes = axes.flatten()
    
    X_clean = df_clean['Licenses'].values.reshape(-1, 1)
    X_scaled_clean = StandardScaler().fit_transform(X_clean)
    
    for idx, disease in enumerate(top_diseases):
        ax = axes[idx]
        
        y = df_clean[disease].values
        if np.all(y == 0):
            ax.text(0.5, 0.5, f'No data for {disease}', 
                    transform=ax.transAxes, ha='center', va='center')
            continue
        model = LinearRegression()
        model.fit(X_scaled_clean, y)
        X_vals = np.linspace(df_clean['Licenses'].min(), df_clean['Licenses'].max(), 100).reshape(-1, 1)
        X_vals_scaled = StandardScaler().fit_transform(X_vals)
        y_line = model.predict(X_vals_scaled)
    
        ax.scatter(df_clean['Licenses'], y, alpha=0.7, s=80, edgecolors='k', label='Actual')
        ax.plot(X_vals, y_line, 'r-', linewidth=2, 
                label=f'Regression (R²={results_df[results_df["Disease"]==disease]["R2"].values[0]:.3f})')
        

        for i, year in enumerate(df_clean['Year']):
            if i % 4 == 0:  # Label every 4th year
                ax.annotate(str(int(year)), 
                           (df_clean['Licenses'].iloc[i], y[i]),
                           xytext=(5, 5), textcoords='offset points', 
                           fontsize=8, alpha=0.7)
        
        ax.set_xlabel('Number of Licenses', fontsize=11)
        ax.set_ylabel(f'{disease} Cases', fontsize=11)
        disease_stats = results_df[results_df['Disease'] == disease]
        if not disease_stats.empty:
            slope = disease_stats['Slope'].values[0]
            p_value = disease_stats['P_Value'].values[0]
            ax.set_title(f'{disease} vs Licenses\nβ={slope:.3f}, p={p_value:.4f}')
        
        ax.legend()
        ax.grid(True, alpha=0.3)
    
    plt.suptitle('Top 4 Diseases by Regression Fit with Licenses', fontsize=16, fontweight='bold', y=1.02)
    plt.tight_layout()
    plt.savefig('regression_top_diseases.png', dpi=300, bbox_inches='tight')
    plt.show()
else:
    print("results_df not found. Run regression analysis first.")

In [ ]:
print("\n" + "=" * 60)
print(" MULTIPLE REGRESSION: Predicting Total Cases")
print("=" * 60)

df_clean = df.fillna(0)

X_multi = df_clean[['Licenses']].copy()
if X_multi['Licenses'].isnull().any():
    print("⚠️ Warning: Licenses column contains NaN after cleaning")
    X_multi['Licenses'] = X_multi['Licenses'].fillna(X_multi['Licenses'].mean())
X_multi['Year_Normalized'] = (df_clean['Year'] - df_clean['Year'].mean()) / (df_clean['Year'].max() - df_clean['Year'].min())

X_multi['Licenses_Squared'] = X_multi['Licenses'] ** 2
if 'Total_Cases' not in df_clean.columns:
    disease_cols_clean = [col for col in df_clean.columns if col not in ['Year', 'Licenses']]
    y_multi = df_clean[disease_cols_clean].sum(axis=1).values
else:
    y_multi = df_clean['Total_Cases'].values
print(f"Checking for NaN in features:")
for col in X_multi.columns:
    nan_count = X_multi[col].isnull().sum()
    print(f"  {col}: {nan_count} NaN values")
X_multi_clean = X_multi.fillna(0)
y_multi_clean = np.nan_to_num(y_multi, nan=0)

scaler_multi = StandardScaler()
X_multi_scaled = scaler_multi.fit_transform(X_multi_clean)

try:
    multi_model = LinearRegression()
    multi_model.fit(X_multi_scaled, y_multi_clean)
    y_multi_pred = multi_model.predict(X_multi_scaled)
    r2_multi = r2_score(y_multi_clean, y_multi_pred)
    mse_multi = mean_squared_error(y_multi_clean, y_multi_pred)
    mae_multi = mean_absolute_error(y_multi_clean, y_multi_pred)
    
    print(f"\nMultiple Regression Results (Total Cases = f(Licenses, Year, Licenses²)):")
    print(f"R²: {r2_multi:.4f}")
    print(f"MSE: {mse_multi:.2f}")
    print(f"MAE: {mae_multi:.2f}")
    print(f"\nCoefficients:")
    feature_names = ['Licenses', 'Year_Normalized', 'Licenses_Squared']
    for i, (name, coef) in enumerate(zip(feature_names, multi_model.coef_)):
        print(f"  {name}: {coef:.4f}")
    print(f"  Intercept: {multi_model.intercept_:.2f}")
    try:
        X_multi_sm = sm.add_constant(X_multi_scaled)
        model_multi_sm = sm.OLS(y_multi_clean, X_multi_sm).fit()
        print("\nDetailed Regression Summary (first few lines):")
        print(model_multi_sm.summary().tables[1])
    except:
        print("\n⚠️ Could not generate detailed statsmodels summary")
        
except Exception as e:
    print(f"\n❌ Error in multiple regression: {e}")
    print("Trying simpler approach...")
    X_simple = df_clean['Licenses'].values.reshape(-1, 1)
    y_simple = y_multi_clean   
    simple_model = LinearRegression()
    simple_model.fit(X_simple, y_simple)
    r2_simple = simple_model.score(X_simple, y_simple)
    
    print(f"\nSimple Linear Regression (Total Cases ~ Licenses):")
    print(f"R²: {r2_simple:.4f}")
    print(f"Slope: {simple_model.coef_[0]:.4f}")

In [2]:
print("\n" + "=" * 60)
print(" TIME SERIES REGRESSION: Impact of Previous Year's Licenses")
print("=" * 60)


df_clean = df.fillna(0)

if 'Total_Cases' not in df_clean.columns:
    disease_cols_lag = [col for col in df_clean.columns if col not in ['Year', 'Licenses']]
    df_clean['Total_Cases'] = df_clean[disease_cols_lag].sum(axis=1)

df_lag = df_clean.copy()
for col in ['Licenses', 'Total_Cases']:
    df_lag[f'{col}_lag1'] = df_lag[col].shift(1)

df_lag_clean = df_lag.dropna().copy()

X_lag = df_lag_clean[['Licenses_lag1']]
y_lag = df_lag_clean['Total_Cases']
if len(X_lag) > 0 and len(y_lag) > 0:
    lag_model = LinearRegression()
    lag_model.fit(X_lag, y_lag)
    y_lag_pred = lag_model.predict(X_lag)
    
    r2_lag = r2_score(y_lag, y_lag_pred)
    lag_slope = lag_model.coef_[0]
    lag_intercept = lag_model.intercept_
    
    print(f"\nRegression: Current Year Total Cases = f(Previous Year Licenses)")
    print(f"Slope: {lag_slope:.4f}")
    print(f"Intercept: {lag_intercept:.2f}")
    print(f"R²: {r2_lag:.4f}")
    print(f"Interpretation: For each additional license last year, total cases increase by {lag_slope:.2f} this year")
    plt.figure(figsize=(10, 6))
    plt.scatter(X_lag, y_lag, alpha=0.7, s=100, edgecolors='k', label='Actual Data')
   
    x_line = np.array([[X_lag.min().iloc[0]], [X_lag.max().iloc[0]]])
    y_line = lag_model.predict(x_line)
    plt.plot(x_line, y_line, 'r-', linewidth=2, label=f'Regression (R²={r2_lag:.3f})')
    
    plt.xlabel('Previous Year Licenses', fontsize=12)
    plt.ylabel('Current Year Total Cases', fontsize=12)
    plt.title('Time Series Regression: Lag Effect of Licenses on Disease Cases', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('time_series_lag_regression.png', dpi=300)
    plt.show()
    
else:
    print("❌ Not enough data for lag regression")


 TIME SERIES REGRESSION: Impact of Previous Year's Licenses


NameError: name 'df' is not defined

In [3]:
print("\n" + "=" * 60)
print(" REGULARIZED REGRESSION: Ridge & Lasso Comparison")
print("=" * 60)
df_clean = df.fillna(0)
if 'Total_Cases' not in df_clean.columns:
    actual_disease_cols = [col for col in df_clean.columns if col not in ['Year', 'Licenses', 'Cases_per_License', 'Production_Index']]
    df_clean['Total_Cases'] = df_clean[actual_disease_cols].sum(axis=1)
    disease_cols_clean = actual_disease_cols
else:
    disease_cols_clean = [col for col in disease_cols if col not in ['Total_Cases', 'Cases_per_License', 'Production_Index']]
X_all = df_clean[disease_cols_clean].copy()
y_all = df_clean['Total_Cases'].values

print(f"Using {len(disease_cols_clean)} diseases as predictors")
print(f"Sample size: {len(X_all)} years")

if X_all.isnull().any().any():
    print("⚠️ X_all contains NaN, filling with 0")
    X_all = X_all.fillna(0)

if np.isnan(y_all).any():
    print("⚠️ y_all contains NaN, filling with 0")
    y_all = np.nan_to_num(y_all)

scaler_all = StandardScaler()
X_all_scaled = scaler_all.fit_transform(X_all)

ridge = Ridge(alpha=1.0)
ridge.fit(X_all_scaled, y_all)

lasso = Lasso(alpha=0.1, max_iter=5000)  # Increased max_iter for convergence
lasso.fit(X_all_scaled, y_all)

coef_comparison = pd.DataFrame({
    'Disease': disease_cols_clean,
    'Ridge_Coefficient': ridge.coef_,
    'Lasso_Coefficient': lasso.coef_,
    'Absolute_Importance_Ridge': np.abs(ridge.coef_),
    'Absolute_Importance_Lasso': np.abs(lasso.coef_)
})

coef_comparison = coef_comparison.sort_values('Absolute_Importance_Ridge', ascending=False)

print("\nTop Predictors of Total Cases (Regularized Regression):")
print(coef_comparison[['Disease', 'Ridge_Coefficient', 'Lasso_Coefficient']].round(4).to_string(index=False))


print("\n Interpretation:")
print("• Ridge coefficients: All diseases contribute (some penalized)")
print("• Lasso coefficients: Only important diseases selected (others set to 0)")
print("• Positive coefficient: Disease contributes to total cases")
print("• Negative coefficient: Disease inversely related to total cases")


selected_by_lasso = coef_comparison[coef_comparison['Lasso_Coefficient'] != 0]['Disease'].tolist()
print(f"\n Diseases selected by Lasso (non-zero coefficients): {selected_by_lasso}")


 REGULARIZED REGRESSION: Ridge & Lasso Comparison


NameError: name 'df' is not defined

In [4]:
print("\n" + "=" * 60)
print(" POLYNOMIAL REGRESSION: Capturing Non-linear Effects")
print("=" * 60)

from sklearn.preprocessing import PolynomialFeatures
df_clean = df.fillna(0)
disease_example = 'IPN'

if disease_example not in df_clean.columns:
    print(f"⚠️ {disease_example} not found in data")
    # Try another disease
    disease_example = 'CMS' if 'CMS' in df_clean.columns else df_clean.columns[2]

print(f"Analyzing {disease_example} vs Licenses")

X_poly = df_clean['Licenses'].values.reshape(-1, 1)
y_poly = df_clean[disease_example].values

if np.all(y_poly == 0):
    print(f"⚠️ All values for {disease_example} are zero")
    # Find a disease with non-zero values
    for col in df_clean.columns:
        if col not in ['Year', 'Licenses'] and not np.all(df_clean[col] == 0):
            disease_example = col
            y_poly = df_clean[disease_example].values
            print(f"Using {disease_example} instead")
            break
degrees = [1, 2, 3, 4]
poly_results = []

for degree in degrees:
    try:
        poly = PolynomialFeatures(degree=degree)
        X_poly_transformed = poly.fit_transform(X_poly)
        
        model_poly = LinearRegression()
        model_poly.fit(X_poly_transformed, y_poly)
        y_poly_pred = model_poly.predict(X_poly_transformed)
        
        r2_poly = r2_score(y_poly, y_poly_pred)
        poly_results.append({
            'Degree': degree,
            'R2': r2_poly,
            'Model': model_poly,
            'Poly': poly
        })
        
        print(f"Degree {degree}: R² = {r2_poly:.4f}")
    except Exception as e:
        print(f"Degree {degree} failed: {e}")

if poly_results:
    plt.figure(figsize=(12, 6))
    
    X_sorted = np.sort(X_poly, axis=0)
    
    colors = ['red', 'blue', 'green', 'orange']
    
    for idx, result in enumerate(poly_results):
        if idx < len(colors):
            color = colors[idx]
        else:
            color = 'gray'
            
        X_plot = result['Poly'].fit_transform(X_sorted)
        y_plot = result['Model'].predict(X_plot)
        
        plt.plot(X_sorted, y_plot, '--', linewidth=2, color=color,
                 label=f'Degree {result["Degree"]} (R²={result["R2"]:.3f})')
    
    plt.scatter(df_clean['Licenses'], y_poly, s=100, alpha=0.7, 
                edgecolors='k', label='Actual Data', zorder=5)
    
    plt.xlabel('Licenses', fontsize=12)
    plt.ylabel(f'{disease_example} Cases', fontsize=12)
    plt.title(f'Polynomial Regression: {disease_example} vs Licenses', fontsize=14, fontweight='bold')
    plt.legend()
    plt.grid(True, alpha=0.3)
    plt.tight_layout()
    plt.savefig('polynomial_regression.png', dpi=300, bbox_inches='tight')
    plt.show()
    
    best_result = max(poly_results, key=lambda x: x['R2'])
    print(f"\n✅ Best fit: Degree {best_result['Degree']} with R² = {best_result['R2']:.4f}")
else:
    print("❌ No polynomial models were successfully fitted")


 POLYNOMIAL REGRESSION: Capturing Non-linear Effects


ModuleNotFoundError: No module named 'sklearn'

In [5]:
print("\n" + "=" * 60)
print(" REGRESSION DIAGNOSTICS: Residual Analysis")
print("=" * 60)
if 'y_multi' not in locals() or 'y_multi_pred' not in locals():
    print("⚠️ Multiple regression results not found. Running simple regression...")

    df_clean = df.fillna(0)
 
    if 'Total_Cases' not in df_clean.columns:
        disease_cols_diag = [col for col in df_clean.columns if col not in ['Year', 'Licenses']]
        df_clean['Total_Cases'] = df_clean[disease_cols_diag].sum(axis=1)
   
    X_simple = df_clean['Licenses'].values.reshape(-1, 1)
    y_multi = df_clean['Total_Cases'].values
    
    simple_model = LinearRegression()
    simple_model.fit(X_simple, y_multi)
    y_multi_pred = simple_model.predict(X_simple)
    
    print(f"✅ Using simple regression (Total Cases ~ Licenses)")
    print(f"   R² = {simple_model.score(X_simple, y_multi):.4f}")

residuals = y_multi - y_multi_pred

print(f"\nResidual Statistics:")
print(f"  Mean: {residuals.mean():.2f}")
print(f"  Std Dev: {residuals.std():.2f}")
print(f"  Min: {residuals.min():.2f}")
print(f"  Max: {residuals.max():.2f}")

fig, axes = plt.subplots(2, 2, figsize=(12, 10))

axes[0, 0].scatter(y_multi_pred, residuals, alpha=0.7, s=80, edgecolors='k')
axes[0, 0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
axes[0, 0].set_xlabel('Fitted Values', fontsize=11)
axes[0, 0].set_ylabel('Residuals', fontsize=11)
axes[0, 0].set_title('Residuals vs Fitted Values', fontsize=12)
axes[0, 0].grid(True, alpha=0.3)

try:
    stats.probplot(residuals, dist="norm", plot=axes[0, 1])
    axes[0, 1].set_title('Q-Q Plot of Residuals', fontsize=12)
    axes[0, 1].grid(True, alpha=0.3)
except:
    axes[0, 1].text(0.5, 0.5, 'Q-Q Plot Failed', 
                    transform=axes[0, 1].transAxes, ha='center', va='center')
    axes[0, 1].set_title('Q-Q Plot (Failed)', fontsize=12)

df_clean = df.fillna(0)
if len(df_clean) == len(residuals):
    axes[1, 0].scatter(df_clean['Year'], residuals, alpha=0.7, s=80, edgecolors='k')
    axes[1, 0].axhline(y=0, color='r', linestyle='--', alpha=0.5)
    axes[1, 0].set_xlabel('Year', fontsize=11)
    axes[1, 0].set_ylabel('Residuals', fontsize=11)
    axes[1, 0].set_title('Residuals vs Time', fontsize=12)
else:
    axes[1, 0].text(0.5, 0.5, 'Year data mismatch', 
                    transform=axes[1, 0].transAxes, ha='center', va='center')
    axes[1, 0].set_title('Residuals vs Time (Data Issue)', fontsize=12)
axes[1, 0].grid(True, alpha=0.3)

axes[1, 1].hist(residuals, bins=10, edgecolor='black', alpha=0.7)
axes[1, 1].axvline(x=0, color='r', linestyle='--', alpha=0.5)
axes[1, 1].set_xlabel('Residuals', fontsize=11)
axes[1, 1].set_ylabel('Frequency', fontsize=11)
axes[1, 1].set_title('Distribution of Residuals', fontsize=12)
axes[1, 1].grid(True, alpha=0.3)

plt.suptitle('Regression Diagnostics: Residual Analysis', fontsize=16, fontweight='bold', y=1.02)
plt.tight_layout()
plt.savefig('regression_diagnostics.png', dpi=300, bbox_inches='tight')
plt.show()

print("\n✅ Regression Diagnostics Check:")
print("1. Residuals vs Fitted: Should show random scatter (no pattern)")
print("2. Q-Q Plot: Points should follow straight line (normality)")
print("3. Residuals vs Time: Should show no trend over time")
print("4. Histogram: Should be roughly bell-shaped")


 REGRESSION DIAGNOSTICS: Residual Analysis
⚠️ Multiple regression results not found. Running simple regression...


NameError: name 'df' is not defined

In [17]:
print("\n KEY INSIGHTS SUMMARY")
print("=" * 60)

print("\n🔴 CRITICAL FINDINGS:")
print("1. Production increased 42% but disease patterns varied")
print("2. IPN declined dramatically (vaccine success story)")
print("3. CMS increased despite controls (emerging concern)")
print("4. HSMI & PD show mixed patterns")

print("\n🟡 REGRESSION INSIGHTS:")
print("• Licenses explain 30-70% of disease variance")
print("• Each disease has unique relationship with production")
print("• Time effects important (vaccine introductions matter)")

print("\n🟢 RECOMMENDATIONS:")
print("1. Continue IPN vaccination programs")
print("2. Develop better CMS management strategies")
print("3. Monitor density effects on HSMI/PD")
print("4. Regular regression analysis for early warnings")


 KEY INSIGHTS SUMMARY

🔴 CRITICAL FINDINGS:
1. Production increased 42% but disease patterns varied
2. IPN declined dramatically (vaccine success story)
3. CMS increased despite controls (emerging concern)
4. HSMI & PD show mixed patterns

🟡 REGRESSION INSIGHTS:
• Licenses explain 30-70% of disease variance
• Each disease has unique relationship with production
• Time effects important (vaccine introductions matter)

🟢 RECOMMENDATIONS:
1. Continue IPN vaccination programs
2. Develop better CMS management strategies
3. Monitor density effects on HSMI/PD
4. Regular regression analysis for early warnings


In [18]:
try:
    z = np.polyfit(df['Year'], df['Cases_per_License'], 1)
except:
    z = [0, 0] 

summary_stats = pd.DataFrame({
    'Total_Cases_2005_2024': df[disease_cols].sum(),
    'Average_Cases_per_Year': df[disease_cols].mean(),
    'Max_Cases_Year': df.loc[df[disease_cols].idxmax(), 'Year'].values,
    'Max_Cases': df[disease_cols].max(),
    'Correlation_with_Licenses': df[disease_cols].apply(lambda x: x.corr(df['Licenses']))
})

summary_stats.to_csv('disease_summary_statistics.csv')
df.to_csv('processed_salmon_diseases_data.csv', index=False)

import os

all_files = [
    'disease_trends.png',
    'licenses_vs_cases.png',
    'correlation_heatmap.png',
    'disease_contribution.png',
    'disease_summary_statistics.csv',
    'processed_salmon_diseases_data.csv',
    'regression_top_diseases.png',
    'polynomial_regression.png',
    'regression_diagnostics.png',
    'time_series_lag_regression.png'
]

print("\n MAIN ANALYSIS FILES :")
main_files_names = [
    'disease_trends.png',
    'licenses_vs_cases.png', 
    'correlation_heatmap.png',
    'disease_contribution.png',
    'disease_summary_statistics.csv',
    'processed_salmon_diseases_data.csv'
]

for file in main_files_names:
    if os.path.exists(file):
        size = os.path.getsize(file) / 1024  
        print(f"  ✅ {file:40} ({size:.1f} KB)")
    else:
        print(f"  ❌ {file:40} (not found)")

print("\n REGRESSION ANALYSIS FILES:")
regression_files_names = [
    'regression_top_diseases.png',
    'polynomial_regression.png',
    'regression_diagnostics.png',
    'time_series_lag_regression.png'
]

regression_found = 0
for file in regression_files_names:
    if os.path.exists(file):
        size = os.path.getsize(file) / 1024  # Size in KB
        print(f"  ✅ {file:40} ({size:.1f} KB)")
        regression_found += 1
    else:
        print(f"  ❌ {file:40} (not found)")
print("\n" + "=" * 60)
print("📦 DOWNLOAD ")
print("=" * 60)

try:
    import shutil
    import zipfile
    if files_found:
        zip_filename = 'salmon_analysis_results.zip'
        with zipfile.ZipFile(zip_filename, 'w') as zipf:
            for file in files_found:
                zipf.write(file)
        
        zip_size = os.path.getsize(zip_filename) / 1024  # KB
        print(f"✅ File: {zip_filename} ({zip_size:.1f} KB)")
        try:
            from IPython.display import FileLink
            print("📥 Download link: ", end="")
            display(FileLink(zip_filename))
        except:
            print(f"📥 File saved at: {os.path.abspath(zip_filename)}")
    else:
        print("⚠️ No files found to zip")
        
except Exception as e:
    print(f"⚠️ Could not create zip file: {e}")
    print(f"📁 Files are in: {os.getcwd()}")


 MAIN ANALYSIS FILES :
  ✅ disease_trends.png                       (597.8 KB)
  ✅ licenses_vs_cases.png                    (337.0 KB)
  ✅ correlation_heatmap.png                  (370.1 KB)
  ✅ disease_contribution.png                 (227.4 KB)
  ✅ disease_summary_statistics.csv           (0.7 KB)
  ✅ processed_salmon_diseases_data.csv       (1.8 KB)

 REGRESSION ANALYSIS FILES:
  ✅ regression_top_diseases.png              (682.2 KB)
  ✅ polynomial_regression.png                (354.5 KB)
  ✅ regression_diagnostics.png               (412.2 KB)
  ✅ time_series_lag_regression.png           (210.6 KB)

📦 DOWNLOAD 
⚠️ Could not create zip file: name 'files_found' is not defined
📁 Files are in: /kaggle/working
